In [6]:
import random
import csv
import operator


class Submission:
    """
    Uses the function that predicts the users interests for an item to find the predicted top ratings and writes it to a
    file in the format specified by the RecSys challenge.

    So far, the recommendations are a reordering of the given impressions. From the train set, we know that the
    click-out item is one of the impressions in 99,95% of the cases, so this seems like a fair assumption.
    """

    def __init__(self, predict_rating_func):
        self.predict_rating_func = predict_rating_func

    def sort_items(self, user, items):
        """
        Given a user and a list of items, this method predicts the user's rating for these items and return a sorted
        list of the items in descending order of the item's predicted ratings.
        """

        # Generates a dictionary with the items as keys and the predicted rating for the user to the items as the values
        predicted_item_ratings = {}
        for item in items:
            predicted_rating = self.predict_rating_func(user, item)
            predicted_item_ratings[item] = predicted_rating

        # Sort the accommodations by highest predicted rating.
        sorted_list_of_tuples = sorted(predicted_item_ratings.items(), key=operator.itemgetter(1), reverse=True)
        items_sorted = [tuple[0] for tuple in sorted_list_of_tuples]

        return items_sorted

    def write_line(self, user_id, session_id, timestamp, step, recommendations):
        """
        Writes a line in the submission file.
        :param recommendations: The list of recommendations in descending order
        """

        with open("submission.txt", mode="a") as submission:
            recommendation_writer = csv.writer(submission, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            recommendation_writer.writerow([user_id, session_id, timestamp, step, " ".join(recommendations)])

    def write_submission_file(self):
        """
        Generate and writes the file that is submitted to the RecSys challenge.
        """

        with open("data/test.csv") as test:
            reader = csv.reader(test)
            for row in reader:
                action_type = row[4]
                if action_type == "clickout item":
                    reference = row[5]
                    if not reference:
                        # This is the line is missing the click-out item. Generating recommendations for this user.
                        user_id, session_id, timestamp, step = row[0:4]
                        items = row[10].split("|")
                        recommendations = self.sort_items(user_id, items)

                        # Writes the result to file
                        self.write_line(user_id, session_id, timestamp, step, recommendations)


In [7]:
def predicted_rating(user, item):
        """
        Dummy function returning the predicted rating for a user-item pair.
        The function is currently returning a random value.
        TODO: Implement this method.
        """
        return random.random()


sub = Submission(predict_rating_func=predicted_rating)
sub.write_submission_file()